# ARTIGO 05 — Representação do Preço (Kairos Alloy)

Referência: `research/ARTIGO 5 - Representação do Preço para DRL em Criptomoedas - Orçamento de Complexidade e Auditoria Anti-Seleção.md`.

Objetivo deste notebook: comparar runs gerados com diferentes *feature sets* / representações,
mantendo orçamento fixo (d, budget de treino) — o notebook só consome artefatos em `runs/<run_id>/`.


In [ ]:
RUN_IDS = []
RUNS_DIR = "runs"


In [ ]:
from pathlib import Path
import sys

import pandas as pd

cwd = Path.cwd().resolve()
repo_root = cwd if (cwd / "crates").exists() else cwd.parent
notebooks_dir = repo_root / "notebooks"
sys.path.insert(0, str(notebooks_dir / "_lib"))

from runs import load_run, latest_run
from dataframes import read_trades_csv, read_equity_csv

runs_dir = (repo_root / RUNS_DIR).resolve()
if not RUN_IDS:
    rd = latest_run(runs_dir)
    if rd is None:
        raise RuntimeError(f"no runs found under {runs_dir}")
    RUN_IDS = [rd.name]

loaded = [load_run(runs_dir / rid) for rid in RUN_IDS]
loaded

## Summary table


In [ ]:
rows = []
for r in loaded:
    s = r.summary or {}
    rows.append({
        "run_id": r.run_id,
        "bars_processed": s.get("bars_processed"),
        "trades": s.get("trades"),
        "net_profit": s.get("net_profit"),
        "sharpe": s.get("sharpe"),
        "max_drawdown": s.get("max_drawdown"),
    })
pd.DataFrame(rows).sort_values(by=["sharpe"], ascending=False)

## Reason sanity

Mesmo em ARTIGO 05 (endógeno), `reason` ajuda a auditar políticas e a detectar degenerescência (ex.: sempre o mesmo motivo).


In [ ]:
for r in loaded:
    if r.trades_csv is None:
        continue
    tr = read_trades_csv(r.trades_csv)
    vc = tr["reason"].fillna("").value_counts().head(10)
    print("\n===", r.run_id)
    print(vc.to_string())

## TODO (ARTIGO 05)

- Quando `manifest.json` existir, agrupar por `representation_id` e garantir orçamento fixo.
- Exportar tabelas/figuras: Δ vs R0, placebos P0/P1, estabilidade por janela/ano.
